# Reading JSON Files

1. JSON stands for JavaScript Object Notation and was designed as a way for applications and systems to exchage
    data between each other.
2. JSON is a light-weight text based file format that is very human readable and has the ability to represent data in 
    a nested format unlike a typical CSV file

# Working with Structured Operations

### Reading a JSON File 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, FloatType, DateType, BooleanType, StructField, StructType,IntegerType,StringType

In [2]:
spark = SparkSession.builder.appName("ReadinJson").getOrCreate()

In [3]:
persons_schema = StructType([
    StructField("id",IntegerType(),True),
    StructField("first_name",StringType(),True),
    StructField("last_name",StringType(),True),
    StructField("fav_movies",ArrayType(StringType()),True),
    StructField("salary",FloatType(),True),
    StructField("image_url",StringType(),True),
    StructField("date_of_birth",DateType(),True),
    StructField("active",BooleanType(),True)
])

In [4]:
data = spark.read.json("persons.json",persons_schema,multiLine=True)

In [5]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- fav_movies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- salary: float (nullable = true)
 |-- image_url: string (nullable = true)
 |-- date_of_birth: date (nullable = true)
 |-- active: boolean (nullable = true)



In [6]:
data.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|
|  3|       Max|   Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|
|  6|    Oswald| Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|   Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|  Go

In [7]:
data.show(10, truncate=False)

+---+----------+---------+-------------------------------------------------------------+-------+-----------------------------------------------+-------------+------+
|id |first_name|last_name|fav_movies                                                   |salary |image_url                                      |date_of_birth|active|
+---+----------+---------+-------------------------------------------------------------+-------+-----------------------------------------------+-------------+------+
|1  |Drucy     |Poppy    |[I giorni contati]                                           |1463.36|http://dummyimage.com/126x166.png/cc0000/ffffff|1991-02-16   |true  |
|2  |Emelyne   |Blaza    |[Musketeer, The, Topralli]                                   |3006.04|http://dummyimage.com/158x106.bmp/cc0000/ffffff|1991-11-02   |false |
|3  |Max       |Rettie   |[The Forgotten Space, Make It Happen]                        |1422.88|http://dummyimage.com/237x140.jpg/ff4444/ffffff|1990-03-03   |false |
|4  

# Columns and Expressions

In [8]:
from pyspark.sql.functions import col, expr

In [9]:
data.select(col("first_name"),col("last_name"),col("date_of_birth")).show(5)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
|    Ilario|     Kean|   1987-06-09|
|     Toddy|   Drexel|   1992-10-28|
+----------+---------+-------------+
only showing top 5 rows



In [10]:
data.select(expr("first_name"),expr("last_name"),expr("date_of_birth")).show(5)

+----------+---------+-------------+
|first_name|last_name|date_of_birth|
+----------+---------+-------------+
|     Drucy|    Poppy|   1991-02-16|
|   Emelyne|    Blaza|   1991-11-02|
|       Max|   Rettie|   1990-03-03|
|    Ilario|     Kean|   1987-06-09|
|     Toddy|   Drexel|   1992-10-28|
+----------+---------+-------------+
only showing top 5 rows



### pyspark.sql.functions provide two functions concat() and concat_ws() to concatenate DataFrame multiple columns into a single column

In [11]:
from pyspark.sql.functions import concat_ws

In [12]:
data.select(concat_ws(' ', col("first_name"),col("last_name")).alias("full_name"),
            col("salary"),
            (col("salary")* 0.10 + col("salary")).alias("salary_increase")).show(10)

+----------------+-------+------------------+
|       full_name| salary|   salary_increase|
+----------------+-------+------------------+
|     Drucy Poppy|1463.36|1609.6959838867188|
|   Emelyne Blaza|3006.04|  3306.64404296875|
|      Max Rettie|1422.88|1565.1680053710938|
|     Ilario Kean|3561.36|3917.4961181640624|
|    Toddy Drexel|4934.87|  5428.35712890625|
| Oswald Petrolli|1153.23| 1268.552978515625|
|   Adrian Clarey|1044.73| 1149.202978515625|
|Dominica Goodnow|1147.76|1262.5360107421875|
|   Emory Slocomb|1082.11|1190.3209838867188|
|   Jeremias Bode|3472.63|  3819.89287109375|
+----------------+-------+------------------+
only showing top 10 rows



In [13]:
data.select(concat_ws(' ', col("first_name"),col("last_name")).alias("full_name"),
            col("salary"),
            expr("salary * 0.10 + salary").alias("salary_increase")).show()

+------------------+-------+------------------+
|         full_name| salary|   salary_increase|
+------------------+-------+------------------+
|       Drucy Poppy|1463.36|1609.6959838867188|
|     Emelyne Blaza|3006.04|  3306.64404296875|
|        Max Rettie|1422.88|1565.1680053710938|
|       Ilario Kean|3561.36|3917.4961181640624|
|      Toddy Drexel|4934.87|  5428.35712890625|
|   Oswald Petrolli|1153.23| 1268.552978515625|
|     Adrian Clarey|1044.73| 1149.202978515625|
|  Dominica Goodnow|1147.76|1262.5360107421875|
|     Emory Slocomb|1082.11|1190.3209838867188|
|     Jeremias Bode|3472.63|  3819.89287109375|
|    Timothy Ervine|1147.61|1262.3709838867187|
|    Leanora Gooder|1327.02| 1459.722021484375|
|   Claiborn Denham|2623.33|   2885.6630859375|
|  Ambrosi Vidineev|4550.88|  5005.96787109375|
|Feodor Nancekivell|2218.46|  2440.30595703125|
|  Margaux Archbold|1013.75|          1115.125|
|   Balduin Elstone|2302.26|2532.4860107421873|
|    Alfie Hatliffe| 3893.1| 4282.410107

# Filter and Where Conditions

### Learn to filter data using the "FILTER", "WHERE" functions. FYI, these are built in DataFrame API functions such as "SELECT, ORDER BY, GROUP BY, FILTER, WHERE"

In [14]:
data.filter("salary <= 3000").show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|      Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  3|       Max|     Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|
|  6|    Oswald|   Petrolli|[Wing and the Thi...|1153.23|http://dummyimage...|   1986-09-02| false|
|  7|    Adrian|     Clarey|[Walking Tall, Pa...|1044.73|http://dummyimage...|   1971-08-24| false|
|  8|  Dominica|    Goodnow|    [Hearts Divided]|1147.76|http://dummyimage...|   1973-08-27| false|
|  9|     Emory|    Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 11|   Timothy|     Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|


In [15]:
data.where((col("salary") <= 3000)& (col("active") == True)).show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|
|  9|     Emory|  Slocomb|[Snake and Crane ...|1082.11|http://dummyimage...|   1974-06-08|  true|
| 16|   Margaux| Archbold|[And Now a Word f...|1013.75|http://dummyimage...|   1988-07-29|  true|
| 26|     Clive|      Lax|             [Rabid]|2126.87|http://dummyimage...|   1981-10-26|  true|
| 33|  Sherline|  Primett|   [Jungle Fighters]|2309.39|http://dummyimage...|   1972-07-23|  true|
| 34|     Davis|    Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 37|    Carlen|  Sharply|[Dr. Jekyll and M...|2051.85|http://dummyimage...|   2002-06-01|  true|
| 40|    Jordan|   L

In [16]:
from pyspark.sql.functions import year

In [17]:
data.filter((year("date_of_birth") == 2000) | (year("date_of_birth") == 1989)).show(10)

+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| id|first_name|  last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+-----------+--------------------+-------+--------------------+-------------+------+
| 14|   Ambrosi|   Vidineev|[Wall Street: Mon...|4550.88|http://dummyimage...|   1989-07-20|  true|
| 15|    Feodor|Nancekivell|   [Monsoon Wedding]|2218.46|http://dummyimage...|   2000-10-07| false|
| 18|     Alfie|   Hatliffe|     [Lord of Tears]| 3893.1|http://dummyimage...|   1989-06-21|  true|
| 25|     Kelcy|     Wogdon|    [Iron Mask, The]|4512.51|http://dummyimage...|   2000-10-20|  true|
| 32|      Redd|   Akenhead|[Century of the D...| 2470.9|http://dummyimage...|   2000-06-05| false|
| 34|     Davis|      Pinks|          [Hounddog]|1337.14|http://dummyimage...|   1989-07-27|  true|
| 61|    Shanna|    Samples|[Thomas in Love (...| 2703.0|http://dummyimage...|   1989-07-07| false|


In [18]:
from pyspark.sql.functions import array_contains

In [19]:
data.where(array_contains(data.fav_movies,"Land of the Lost")).show()

+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies| salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+
| 11|   Timothy|   Ervine|[Land of the Lost...|1147.61|http://dummyimage...|   1971-06-02| false|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+



# Distict, Drop, Duplicates, Order By

Learn how to get unique set of rows, drop duplicates and sort/ order the dataframe

In [20]:
from pyspark.sql.functions import count, desc

In [21]:
data.select("active").show(10)

+------+
|active|
+------+
|  true|
| false|
| false|
|  true|
|  true|
| false|
| false|
| false|
|  true|
|  true|
+------+
only showing top 10 rows



In [22]:
data.select("active").distinct().show()

+------+
|active|
+------+
|  true|
| false|
+------+



In [23]:
data.select(col("first_name"),
           year(col("date_of_birth")).alias("year"),
           col("active")).orderBy("year","first_name").show(10)

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|    Adrian|1971| false|
|   Feodora|1971|  true|
|       Sky|1971| false|
|   Timothy|1971| false|
|    Lucita|1972|  true|
|      Rodi|1972| false|
|  Sherline|1972|  true|
|     Toddy|1972|  true|
|  Dominica|1973| false|
|    Kelila|1973|  true|
+----------+----+------+
only showing top 10 rows



In [24]:
dropped_df = data.select(col("first_name"),
                        year(col("date_of_birth")).alias("year"),
                        col("active")).dropDuplicates(["year","active"]).orderBy("year", "first_name")

In [25]:
dropped_df.show()

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|    Adrian|1971| false|
|   Feodora|1971|  true|
|      Rodi|1972| false|
|  Sherline|1972|  true|
|  Dominica|1973| false|
|    Kelila|1973|  true|
|   Balduin|1974| false|
|     Emory|1974|  true|
|    Janean|1975|  true|
|       Bev|1976|  true|
| Franciska|1976| false|
|     Johny|1977| false|
|    Daveta|1978| false|
|   Guthrie|1978|  true|
|      Maxi|1979| false|
|   Melinda|1979|  true|
|    Carter|1980| false|
|   Loralyn|1980|  true|
|     Clive|1981|  true|
|   Leanora|1981| false|
+----------+----+------+
only showing top 20 rows



In [26]:
data.select(col("first_name"),
                        year(col("date_of_birth")).alias("year"),
                        col("active")).orderBy("year",ascending=False).show(10)

+----------+----+------+
|first_name|year|active|
+----------+----+------+
|     Daron|2002|  true|
|    Virgie|2002|  true|
|    Carlen|2002|  true|
|   Lorilee|2002| false|
|    Maxine|2001| false|
|    Feodor|2000| false|
|     Kelcy|2000|  true|
|  Annabell|2000|  true|
|      Redd|2000| false|
|     Jobie|2000| false|
+----------+----+------+
only showing top 10 rows



# Rows and Union

How to crate indivisual rows and make dataframe out of the rows, and use the union transforamtion to combine dataframes

In [27]:
from pyspark.sql import Row

In [28]:
person_row = Row(101, "Robert","Ownes",
                 ["Men in Black III","Home Alone"],4300.64,"http://someimage.com","1964-08-18",True)

In [29]:
person_rows_list = [Row(102, "Kenny","Bobien",
                     ["Men in Black III","Home Alone"],4300.64,"http://someimage.com","1964-08-18",True),
                   Row(103, "Sara","Devine",
                     ["Men in Black III","Home Alone"],4300.64,"http://someimage.com","1964-08-18",True)]

In [30]:
person_rows_list.append(person_row)

In [31]:
print(person_rows_list)

[<Row(102, 'Kenny', 'Bobien', ['Men in Black III', 'Home Alone'], 4300.64, 'http://someimage.com', '1964-08-18', True)>, <Row(103, 'Sara', 'Devine', ['Men in Black III', 'Home Alone'], 4300.64, 'http://someimage.com', '1964-08-18', True)>, <Row(101, 'Robert', 'Ownes', ['Men in Black III', 'Home Alone'], 4300.64, 'http://someimage.com', '1964-08-18', True)>]


In [32]:
person_row[1]

'Robert'

In [33]:
new_person_df = spark.createDataFrame(person_rows_list,["id","first_name","last_name","fav_movies","salary","image_url","date_of_birth","active"])

In [34]:
add_person_df = data.union(new_person_df)
add_person_df.sort(desc("id")).show()

+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
| id|first_name|last_name|          fav_movies|            salary|           image_url|date_of_birth|active|
+---+----------+---------+--------------------+------------------+--------------------+-------------+------+
|103|      Sara|   Devine|[Men in Black III...|           4300.64|http://someimage.com|   1964-08-18|  true|
|102|     Kenny|   Bobien|[Men in Black III...|           4300.64|http://someimage.com|   1964-08-18|  true|
|101|    Robert|    Ownes|[Men in Black III...|           4300.64|http://someimage.com|   1964-08-18|  true|
|100|    Virgie| Domanski|[Horseman, The, S...| 2165.929931640625|http://dummyimage...|   2002-01-05|  true|
| 99|   Rozalie|   Wannop|[Suddenly, The No...|1259.6400146484375|http://dummyimage...|   1997-03-25| false|
| 98|     Davin|     Labb|[Viva Riva!, Kill...| 1452.739990234375|http://dummyimage...|   1988-01-27|  true|
| 97|      Rodi|   

# Adding,Renaming and Dropping Columns

In [35]:
from pyspark.sql.functions import round

In [36]:
aug_persons_df1 = data.withColumn("salary_increase",expr("salary * 0.10 + salary"))

In [37]:
aug_persons_df1.columns

['id',
 'first_name',
 'last_name',
 'fav_movies',
 'salary',
 'image_url',
 'date_of_birth',
 'active',
 'salary_increase']

In [38]:
aug_persons_df2 = (aug_persons_df1
                    .withColumn("birth_year",year("date_of_birth"))
                    .withColumnRenamed("fav_movies","movies")
                    .withColumn("salary_x10",round(col("salary_increase"),2))
                    .drop("salary_increase"))

In [39]:
aug_persons_df2.show(10)

+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
| id|first_name|last_name|              movies| salary|           image_url|date_of_birth|active|birth_year|salary_x10|
+---+----------+---------+--------------------+-------+--------------------+-------------+------+----------+----------+
|  1|     Drucy|    Poppy|  [I giorni contati]|1463.36|http://dummyimage...|   1991-02-16|  true|      1991|    1609.7|
|  2|   Emelyne|    Blaza|[Musketeer, The, ...|3006.04|http://dummyimage...|   1991-11-02| false|      1991|   3306.64|
|  3|       Max|   Rettie|[The Forgotten Sp...|1422.88|http://dummyimage...|   1990-03-03| false|      1990|   1565.17|
|  4|    Ilario|     Kean|[Up Close and Per...|3561.36|http://dummyimage...|   1987-06-09|  true|      1987|    3917.5|
|  5|     Toddy|   Drexel|[Walk in the Clou...|4934.87|http://dummyimage...|   1992-10-28|  true|      1992|   5428.36|
|  6|    Oswald| Petrolli|[Wing and the 

# Working with Missing or Bad Data

how to clean the dataframe and remove missing or bad data

In [40]:
bad_movies_list = [Row(None,None,None),
                  Row(None, None, 2020),
                  Row("Jone Doe","Awesome movie",None),
                  Row(None, "Awesome Movie",2021),
                  Row("Mary Jane",None,2019),
                  Row("vikter Duplaix","Not another teen movie",2001)]

In [41]:
bad_movies_list

[<Row(None, None, None)>,
 <Row(None, None, 2020)>,
 <Row('Jone Doe', 'Awesome movie', None)>,
 <Row(None, 'Awesome Movie', 2021)>,
 <Row('Mary Jane', None, 2019)>,
 <Row('vikter Duplaix', 'Not another teen movie', 2001)>]

In [42]:
bad_movies_columns = ["actor_name","movie_title","produced_year"]

In [43]:
bad_movies_df = spark.createDataFrame(bad_movies_list, schema=bad_movies_columns)

In [44]:
bad_movies_df.show()

+--------------+--------------------+-------------+
|    actor_name|         movie_title|produced_year|
+--------------+--------------------+-------------+
|          null|                null|         null|
|          null|                null|         2020|
|      Jone Doe|       Awesome movie|         null|
|          null|       Awesome Movie|         2021|
|     Mary Jane|                null|         2019|
|vikter Duplaix|Not another teen ...|         2001|
+--------------+--------------------+-------------+



In [45]:
bad_movies_df.na.drop().show()

+--------------+--------------------+-------------+
|    actor_name|         movie_title|produced_year|
+--------------+--------------------+-------------+
|vikter Duplaix|Not another teen ...|         2001|
+--------------+--------------------+-------------+



In [46]:
bad_movies_df.na.drop("any").show()

+--------------+--------------------+-------------+
|    actor_name|         movie_title|produced_year|
+--------------+--------------------+-------------+
|vikter Duplaix|Not another teen ...|         2001|
+--------------+--------------------+-------------+



In [47]:
bad_movies_df.na.drop("all").show()

+--------------+--------------------+-------------+
|    actor_name|         movie_title|produced_year|
+--------------+--------------------+-------------+
|          null|                null|         2020|
|      Jone Doe|       Awesome movie|         null|
|          null|       Awesome Movie|         2021|
|     Mary Jane|                null|         2019|
|vikter Duplaix|Not another teen ...|         2001|
+--------------+--------------------+-------------+



In [48]:
bad_movies_df.filter(col("actor_name").isNull() != True).show()

+--------------+--------------------+-------------+
|    actor_name|         movie_title|produced_year|
+--------------+--------------------+-------------+
|      Jone Doe|       Awesome movie|         null|
|     Mary Jane|                null|         2019|
|vikter Duplaix|Not another teen ...|         2001|
+--------------+--------------------+-------------+



In [49]:
bad_movies_df.filter(col("actor_name").isNull() != False).show()

+----------+-------------+-------------+
|actor_name|  movie_title|produced_year|
+----------+-------------+-------------+
|      null|         null|         null|
|      null|         null|         2020|
|      null|Awesome Movie|         2021|
+----------+-------------+-------------+



In [50]:
bad_movies_df.describe("produced_year").show()

+-------+-----------------+
|summary|    produced_year|
+-------+-----------------+
|  count|                4|
|   mean|          2015.25|
| stddev|9.535023160258536|
|    min|             2001|
|    max|             2021|
+-------+-----------------+



In [51]:
bad_movies_df.describe("actor_name").show()

+-------+--------------+
|summary|    actor_name|
+-------+--------------+
|  count|             3|
|   mean|          null|
| stddev|          null|
|    min|      Jone Doe|
|    max|vikter Duplaix|
+-------+--------------+



## Working with User Defined Functions

In [52]:
from pyspark.sql.functions import udf

In [53]:
student_list = [("Joe", 85),
               ("Jane", 90),
               ("Mary", 55)]

In [54]:
students_columns = ["name","score"]

In [55]:
students_df = spark.createDataFrame(student_list, schema=students_columns)

In [56]:
students_df.show()

+----+-----+
|name|score|
+----+-----+
| Joe|   85|
|Jane|   90|
|Mary|   55|
+----+-----+



In [57]:
def letterGrade(score:int):
    grade = ''
    if score > 100:
        grade = "cheating"
    elif score >= 90:
        grade = "A"
    elif score >=80:
        grade = "B"
    elif score >= 70:
        grade = "C"
    else:
        grade = "Fail"
    return grade

In [58]:
print(letterGrade(75))

C


In [59]:
letterGradeUDF =udf(letterGrade)

In [60]:
students_df.select("name", "score", letterGradeUDF(col("score")).alias("grade")).show()

+----+-----+-----+
|name|score|grade|
+----+-----+-----+
| Joe|   85|    B|
|Jane|   90|    A|
|Mary|   55| Fail|
+----+-----+-----+

